# Run parcel tracking with tracer

In [1]:
import numpy as np
from datetime import timedelta as delta
from datetime import datetime
import xarray as xr
from parcels import FieldSet, Variable, ParticleSet, JITParticle, AdvectionRK4, AdvectionRK4_3D, ErrorCode, Field
import glob

%matplotlib inline

#### Parameters

In [2]:
year = 2002
land_color = '#a9a7a2'
jmin, jmax = 159, 799
imin, imax = 1139, 2179

#### Load files

In [3]:
# Open particle initialization location file:
ini_loc = xr.open_dataset('/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/western-PC-transect-parcels.nc')
parcel_lons = ini_loc['longitude'].values
parcel_lats = ini_loc['latitude'].values
parcel_deps = ini_loc['depth'].values

#### General setup

In [4]:
# Mesh mask
mesh_mask = '/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc'

# ANHA12 NEMO grid U,V,W files
folder_ANHA12      = '/data/brogalla/ANHA12/'
ANHA12_gridU_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridU.nc'))[0:20]
ANHA12_gridV_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridV.nc'))[0:20]
ANHA12_gridW_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_5d_gridW_y{year}m??d??.nc'))[0:20]

# ANHA12 NEMO Pb model files
folder_Pb = f'/data/brogalla/run_storage/Pb-new-ini2-20220922/'
files_Pb  = sorted(glob.glob(f'{folder_Pb}ANHA12_EXH006_5d_{year}0101_{year}1231_ptrc_T_{year}*'))[0:20]

# Create timestamps from file dates:
time_stamps = []
for n, file in enumerate(ANHA12_gridU_files):
    dateU  = datetime.strptime(file[36:47],'y%Ym%md%d')
    dateV  = datetime.strptime(ANHA12_gridV_files[n][36:47],'y%Ym%md%d')
    dateW  = datetime.strptime(ANHA12_gridW_files[n][45:56],'y%Ym%md%d')
    datePb = datetime.strptime(files_Pb[n][99:107],'%Y%m%d')
    
    if (dateU != dateV) | (dateU != dateW) | (dateU != datePb):
        print('Dates not equal!')
    
    # double-check that the date is the same for U, V, and W files
    time_stamps.append(np.datetime64(f'{dateU.year:02}-{dateU.month:02}-{dateU.day:02}'))

timestamps = np.expand_dims(np.array(time_stamps), axis=1)

In [5]:
filenames = {'U': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridU_files},
             'V': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridV_files},
             'W': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridW_files},
             'Pb': {'lon': files_Pb[0], 'lat': files_Pb[0], 'depth': files_Pb[0], 'data': files_Pb}}

dimensions = {'U': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'V': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'W': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'Pb': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'deptht'}}

variables = {'U': 'vozocrtx',
             'V': 'vomecrty',
             'W': 'vovecrtz',
             'Pb': 'dissolpb'}

indices_subset = {'U':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'V':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'W':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'Pb': {'lon': range(0,640)     , 'lat': range(0,1040)    , 'depth': range(0,50)}}

# needed to provide time as timestamps in order to avoid time origin type issue
fieldset  = FieldSet.from_nemo(filenames, variables, dimensions, \
                               timestamps=timestamps, \
                               allow_time_extrapolation=False, \
                               indices=indices_subset)

Particle set with starting locations specified:

In [12]:
release_time  = 0
release_lon   = [-130, -145, -70, -55] # east
release_lat   = [75, 74, 75, 65]       # north
pset = ParticleSet(fieldset=fieldset, pclass=PbParticle, lon=release_lon, lat=release_lat, time=0)

#### Classes and functions

In [7]:
# define new particle class for Pb
class PbParticle(JITParticle): 
    dissolvedPb = Variable('dissolpb', initial=fieldset.Pb)  # Variable 'dissolpb' initialised by sampling the dissolved Pb field

In [8]:
def DeleteParticle(particle, fieldset, time):
    print("Particle [%d] lost !! (%g %g %g %g)" % (particle.id, particle.lon, particle.lat, particle.depth, particle.time))
    particle.delete()

In [9]:
def SamplePb(particle, fieldset, time):
         particle.dissolpb = fieldset.Pb[particle.time, particle.depth, particle.lat, particle.lon]

#### Run

The issue relates to adding the Pb sampling kernel, since running without it the particles are fine:

In [13]:
output_file = pset.ParticleFile(name="/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr", outputdt=delta(hours=12))

# particles are bumping against land and being deleted, if I instead just want them to keep moving:
pset.execute(AdvectionRK4_3D + pset.Kernel(SamplePb), 
             runtime     = delta(hours=24*10), 
             dt          = delta(hours=2),
             output_file = output_file,
             recovery    = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

# dt      --- the timestep of the kernel
# runtime --- the total length of the run

INFO: Compiled ArrayPbParticleAdvectionRK4_3DSamplePb ==> /tmp/parcels-2672/libafb714e8dd33c3ebcc7455ddd13ee995_0.so


Particle [0] lost !! (-130 75 0.494025 0)
Particle [1] lost !! (-145 74 0.494025 0)
Particle [3] lost !! (-55 65 0.494025 0)


INFO: Output files are stored in /ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 864000.0/864000.0 [00:05<00:00, 157364.94it/s]


Without Pb:

In [14]:
output_file = pset.ParticleFile(name="/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr", outputdt=delta(hours=12))

# particles are bumping against land and being deleted, if I instead just want them to keep moving:
pset.execute(AdvectionRK4_3D, 
             runtime     = delta(hours=24*10), 
             dt          = delta(hours=2),
             output_file = output_file,
             recovery    = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

# dt      --- the timestep of the kernel
# runtime --- the total length of the run

INFO: Compiled ArrayPbParticleAdvectionRK4_3D ==> /tmp/parcels-2672/lib26a5e118733e59286960bab7af2293e9_0.so
INFO: Output files are stored in /ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 864000.0/864000.0 [00:04<00:00, 197858.34it/s]
